# <center> N-Body gravitacional em Python x Python NumPy x </center>
# <center> Python Numba CPU x Python Numba CPU // </center>
## <center> v1.5.5 - 22/11/2018 a 15/02/2019 </center>
## <center> $M = 500$ passos, $dt = 0.001$, $N = 1000$ corpos em cubo homogêneo</center>

### Autoria da v1.0, v1.3, v1.4 : Flávio Manoel Santos Hemerli
#### Revisão v1.1, v1.2, v1.3.1, v1.4.1, v1.4.6-9, v1.5.0-5 : Roberto Colistete Júnior

Computador do prof. Roberto Colistete Jr. :  
**Notebook Dell XPS 15** com CPU [**Intel Core i7 2670QM**](https://ark.intel.com/products/53469/Intel-Core-i7-2670QM-Processor-6M-Cache-up-to-3-10-GHz-) 2,2-3,1 GHz 4 cores/8 threads 6MB cache com GPU integrada Intel HD Graphics 3000 & GPU dedicada [**NVidia GeForce GPU GT 540M**](https://www.geforce.com/hardware/notebook-gpus/geforce-gt-540m/specifications) 2GB 96 cores CC2.1.  
Manjaro KDE 18.0.2 64 bits, kernel 4.20.3, gcc 8.2.1, driver NVidia 390.87, CUDA 8.0.  
Anaconda 3 2018-12, Python 3.7.1, NumPy 1.15.4, PyCUDA 2018.1.1 (Python 2.7.15), Numba 0.41.0 com CUDAToolkit 8.0.

-> Computador do prof. Roberto Colistete Jr. :  
**Notebook Dell G3 15 3579** com CPU [**Intel Core i7 8750H**](https://ark.intel.com/products/134906/Intel-Core-i7-8750H-Processor-9M-Cache-up-to-4-10-GHz-) Kaby Lake 2,2-4,1 GHz 6 cores/12 threads 9MB cache com GPU integrada Intel UHD Graphics 630 & GPU dedicada [**NVidia GeForce GTX 1050 Ti Mobile**](https://www.nvidia.com/en-us/geforce/products/10series/laptops/) 4GB 768 cores CC6.1.   
Manjaro KDE 18.0.2 64 bits, kernel 4.20.1, gcc 8.2.1, driver NVidia 415.25, CUDA 10.0.  
Anaconda 3 2018-12, Python 3.7.1, NumPy 1.15.4, PyCUDA 2018.1.1 (Python 2.7.15), Numba 0.41.0 com CUDAToolkit 9.2.

Computador do aluno Flávio Manoel Santos Hemerli:   
**Notebook Lenovo Ideapad 320** com CPU [**Intel Core i5 7200U**](https://ark.intel.com/pt-br/products/95443/Intel-Core-i5-7200U-Processor-3M-Cache-up-to-3-10-GHz-) 2,5-3,1 GHz 2 cores/4 threads 3MB cache com GPU integrada Intel HD Graphics 620. 8GB RAM DDR4 2133MHz Dual Channel.  
Manjaro Deepin 18.0 64 bits. Anaconda 3 2018-12, Python 3.7.1, NumPy 1.15.4, Numba 0.41.0

### Inicialização

In [1]:
import math
import numpy as np
from timeit import default_timer as timer
import numba as nb
from numba import jit, njit, prange

### Informação sobre versão de Linux, CPU :

In [2]:
!uname -a

Linux nautilus9 4.20.7-1-MANJARO #1 SMP PREEMPT Wed Feb 6 21:54:48 UTC 2019 x86_64 GNU/Linux


In [3]:
!lsb_release -a

LSB Version:	n/a
Distributor ID:	ManjaroLinux
Description:	Manjaro Linux
Release:	18.0.2
Codename:	Illyria


In [4]:
!lscpu

Arquitetura:                x86_64
Modo(s) operacional da CPU: 32-bit, 64-bit
Ordem dos bytes:            Little Endian
Tamanhos de endereço:       39 bits physical, 48 bits virtual
CPU(s):                     12
Lista de CPU(s) on-line:    0-11
Thread(s) per núcleo:       2
Núcleo(s) por soquete:      6
Soquete(s):                 1
Nó(s) de NUMA:              1
ID de fornecedor:           GenuineIntel
Família da CPU:             6
Modelo:                     158
Nome do modelo:             Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Step:                       10
CPU MHz:                    3989.018
CPU MHz máx.:               4100,0000
CPU MHz mín.:               800,0000
BogoMIPS:                   4417.00
Virtualização:              VT-x
cache de L1d:               32K
cache de L1i:               32K
cache de L2:                256K
cache de L3:                9216K
CPU(s) de nó0 NUMA:         0-11
Opções:                     fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca c

Versão de gcc :

In [5]:
!gcc --version

gcc (GCC) 8.2.1 20181127
Copyright (C) 2018 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



Versão de Python :

In [6]:
!python --version

Python 3.7.1


Versão de NumPy :

In [7]:
np.__version__

'1.15.4'

Versão de Numba :

In [8]:
nb.__version__

'0.41.0'

In [9]:
nb.config.NUMBA_DEFAULT_NUM_THREADS

12

### Definições de nbody para Python puro, Numba CPU e Numba CPU //

In [10]:
G = 6.67408*1e-11 # define a constante gravitacional
sft = 1e-10 # define a constante de atenuamento para tratamento das singularidades

In [11]:
@njit
def gravitationalNbody_NumbaCPU(r, v, a, m, nobj, ninter, dt):
    for inter in range(ninter):
        for i in range(nobj):
            a[i][0] = 0.0; a[i][1] = 0.0; a[i][2] = 0.0
            for j in range(nobj):
                if i != j:
                    # calculate a_ij acceleration
                    dx = r[j][0] - r[i][0]; dy = r[j][1] - r[i][1]; dz = r[j][2] - r[i][2]
                    dsq = dx*dx + dy*dy + dz*dz + sft*sft
                    gA = (G*m[j])/(dsq*math.sqrt(dsq))
                    # sum a_ij_x to a_i_x, etc
                    a[i][0] += gA*dx; a[i][1] += gA*dy; a[i][2] += gA*dz
            # update the v_i velocity
            v[i][0] += a[i][0]*dt; v[i][1] += a[i][1]*dt; v[i][2] += a[i][2]*dt
        for i in range(nobj):
            # update the r_i position
            r[i][0] += v[i][0]*dt; r[i][1] += v[i][1]*dt; r[i][2] += v[i][2]*dt
    K = 0.0; U = 0.0
    for i in range(nobj):
        # Sum the total kinectic energy
        K += 0.5*m[i]*(v[i][0]**2 + v[i][1]**2 + v[i][2]**2)
        for j in range(nobj):
            if (i < j):
                # Sum the total potential energy
                U += (-G*m[i]*m[j])/math.sqrt((r[j][0] - r[i][0])**2 + (r[j][1] - r[i][1])**2 + 
                                              (r[j][2] - r[i][2])**2 + sft**2)
    return K, U

In [12]:
@njit(parallel=True)
def gravitationalNbody_NumbaCPUparallel(r, v, a, m, nobj, ninter, dt):
    for inter in range(ninter):
        for i in prange(nobj):
            a[i][0] = 0.0; a[i][1] = 0.0; a[i][2] = 0.0
            for j in range(nobj):
                if i != j:
                    # calculate a_ij acceleration
                    dx = r[j][0] - r[i][0]; dy = r[j][1] - r[i][1]; dz = r[j][2] - r[i][2]
                    dsq = dx*dx + dy*dy + dz*dz + sft*sft
                    gA = (G*m[j])/(dsq*math.sqrt(dsq))
                    # sum a_ij_x to a_i_x, etc
                    a[i][0] += gA*dx; a[i][1] += gA*dy; a[i][2] += gA*dz
            # update the v_i velocity
            v[i][0] += a[i][0]*dt; v[i][1] += a[i][1]*dt; v[i][2] += a[i][2]*dt
        for i in prange(nobj):
            # update the r_i position
            r[i][0] += v[i][0]*dt; r[i][1] += v[i][1]*dt; r[i][2] += v[i][2]*dt
    K = 0.0; U = 0.0
    for i in prange(nobj):
        # Sum the total kinectic energy
        K += 0.5*m[i]*(v[i][0]**2 + v[i][1]**2 + v[i][2]**2)
        for j in range(nobj):
            if (i < j):
                # Sum the total potential energy
                U += (-G*m[i]*m[j])/math.sqrt((r[j][0] - r[i][0])**2 + (r[j][1] - r[i][1])**2 + 
                                              (r[j][2] - r[i][2])**2 + sft**2)
    return K, U

In [13]:
@njit
def gravitationalNbody_NumPy_NumbaCPU(r, v, a, m, nobj, ninter, dt):
    for inter in range(ninter):
        # zero the a acceleration
        a = np.zeros((nobj, 3))
        for i in range(nobj):
            # update the a acceleration
            dr = r[i] - r
            dsq = np.sum(dr*dr, axis=1) + sft*sft
            gA = (G*m[i])/(dsq*np.sqrt(dsq))
            a += (dr.T*gA).T
        # update the v velocity
        v += a*dt
        # update the r position
        r += v*dt
    K = np.sum(0.5*m*np.sum(v*v, axis=1))
    U = 0.0
    for i in prange(nobj):
        for j in range(nobj):
            if (i < j):
                # Sum the total potential energy
                U += (-G*m[i]*m[j])/math.sqrt((r[j][0] - r[i][0])**2 + (r[j][1] - r[i][1])**2 + 
                                              (r[j][2] - r[i][2])**2 + sft**2)
    return K, U

In [14]:
@njit(parallel=True)
def gravitationalNbody_NumPy_NumbaCPUparallel(r, v, a, m, nobj, ninter, dt):
    for inter in range(ninter):
        # zero the a acceleration
        a = np.zeros((nobj, 3))
        for i in prange(nobj):
            # update the a acceleration
            dr = r[i] - r
            dsq = np.sum(dr*dr, axis=1) + sft*sft
            gA = (G*m[i])/(dsq*np.sqrt(dsq))
            a += (dr.T*gA).T
        # update the v velocity
        v += a*dt
        # update the r position
        r += v*dt
    K = np.sum(0.5*m*np.sum(v*v, axis=1))
    U = 0.0
    for i in prange(nobj):
        for j in range(nobj):
            if (i < j):
                # Sum the total potential energy
                U += (-G*m[i]*m[j])/math.sqrt((r[j][0] - r[i][0])**2 + (r[j][1] - r[i][1])**2 + 
                                              (r[j][2] - r[i][2])**2 + sft**2)
    return K, U

In [15]:
# n_cube_edge : número de pontos l de cada aresta de um cubo
# ninter : M, número de interações
# dt : dt, intervalo entre interações
def run_nbody(gravitationalNbodyfunction, n_cube_edge=10, ninter = 5, dt = 0.01, 
              flag_array=True, flag_compile=True, flag_verbose=False):
    nbody_version = "1.5.5"

    nobj = n_cube_edge**3 # define o número de objetos na simulação
    if flag_array:
        positions = np.zeros((nobj, 3))   # inicializa uma lista de N posições 3D, nulas inicialmente
        velocities = np.zeros_like(positions)   # inicializa uma lista de N velocidades 3D, nulas inicialmente
        accelerations = np.zeros_like(positions)   # inicializa uma lista de N acelerações 3D, nulas inicialmente
        masses = 1e9*np.ones(nobj)   # inicializa uma lista de N massas, aqui iguais a 10^6 kg
    else:
        positions = [[0., 0., 0.] for i in range(nobj)]   # inicializa uma lista de N de objetos para posição e massa, nulas inicialmente
        velocities = [[0., 0., 0.] for i in range(nobj)]   # inicializa uma lista de N velocidades 3D, nulas inicialmente
        accelerations = [[0., 0., 0.] for i in range(nobj)]   # inicializa uma lista de N acelerações 3D, nulas inicialmente
        masses = [1e9]*nobj   # inicializa uma lista de N massas, aqui iguais a 10^6 kg
    # Cria um cubo de l^3 pontos, onde cada aresta tem l pontos   
    scale = 1.0  # escala do cubo, com comprimento da aresta = scale*n_cube_edge
    n_half_cube_edge = int(n_cube_edge/2)
    for i in range(n_cube_edge):
        for j in range(n_cube_edge):
            for k in range(n_cube_edge):
                positions[k + j*n_cube_edge + i*n_cube_edge*n_cube_edge] = [(i - n_half_cube_edge)*scale, 
                                                                            (j - n_half_cube_edge)*scale,
                                                                            (k - n_half_cube_edge)*scale]                
    if flag_compile:
        # To Numba compile the function 1st time before calling with full data
        positions_backup = positions.copy()
        velocities_backup = velocities.copy()
        accelerations_backup = accelerations.copy()
        print("N-Body (C) Flavio Manoel, v{}".format(nbody_version))
        print("N = {} corpos, dt = {}s, M = {} interações".format(nobj, dt, ninter))
        start = timer()
        Kf, Uf = gravitationalNbodyfunction(positions, velocities, accelerations, masses, nobj, ninter, dt)
        end = timer()
        print("Após {} passos, {:.4f}s, Ef = {:.6f}, Kf = {:.6f}, Uf = {:.6f}".format(ninter, ninter*dt, Kf + Uf, Kf, Uf))
        dt_compile_run = end - start
        print("Código executado em {:.6f} s\n".format(dt_compile_run))
        positions = positions_backup.copy(); velocities = velocities_backup.copy(); accelerations = accelerations_backup.copy()
    else:
        dt_compile_run = 0.0
        
    print("N-Body (C) Flavio Manoel, v{}".format(nbody_version))
    print("N = {} corpos, dt = {}s, M = {} interações".format(nobj, dt, ninter))
    if flag_verbose:
        print("Posições = {}\nVelocidades = {}\nAcelerações = {}\nMassas = {}\n".format(positions, 
              velocities, accelerations, masses))
    Ki, Ui = gravitationalNbodyfunction(positions, velocities, accelerations, masses, nobj, 0, dt)
    print("Ei = {}, Ki = {}, Ui = {}".format(Ki + Ui, Ki, Ui))
    start = timer()
    Kf, Uf = gravitationalNbodyfunction(positions, velocities, accelerations, masses, nobj, ninter, dt)
    end = timer()
    erro_perc = 100.0*(Kf + Uf - (Ki + Ui))/(Ki + Ui)
    print("Após {} passos, {:.4f}s :\nEf = {}, Kf = {}, Uf = {}, erro em E = {}%".format(ninter, 
           ninter*dt, Kf + Uf, Kf, Uf, erro_perc))
    dt_run = end - start
    print("Código executado em {:.6f} s".format(dt_run))
    if flag_verbose:
        print("\nPositions = {}\nVelocities = {}\nAccelerations = {}".format(positions, velocities, 
                                                                             accelerations))
    return dt_compile_run, dt_run, erro_perc

### Teste de nbody em Python puro

In [32]:
t_all_py, t_py, erro_py = run_nbody(gravitationalNbody_NumbaCPU.py_func, n_cube_edge=10, ninter = 500, 
                                    dt = 0.001, flag_array=False, flag_compile=False)

N-Body (C) Flavio Manoel, v1.5.5
N = 1000 corpos, dt = 0.001s, M = 500 interações
Ei = -6221418016861.29, Ki = 0.0, Ui = -6221418016861.29
Após 500 passos, 0.5000s :
Ef = -6221859489860.983, Kf = 211027971915.90527, Uf = -6432887461776.889, erro em E = 0.007096018922002654%
Código executado em 312.768374 s


### Teste de nbody em Python (com NumPy arrays)

In [33]:
t_all_py_a, t_py_a, erro_py_a = run_nbody(gravitationalNbody_NumbaCPU.py_func, n_cube_edge=10, 
                                          ninter = 500, dt = 0.001, flag_compile=False)

N-Body (C) Flavio Manoel, v1.5.5
N = 1000 corpos, dt = 0.001s, M = 500 interações
Ei = -6221418016861.29, Ki = 0.0, Ui = -6221418016861.29
Após 500 passos, 0.5000s :
Ef = -6221859489860.983, Kf = 211027971915.90527, Uf = -6432887461776.889, erro em E = 0.007096018922002654%
Código executado em 1354.979158 s


### Teste de nbody em Python NumPy

In [34]:
t_all_np, t_np, erro_np = run_nbody(gravitationalNbody_NumPy_NumbaCPU.py_func, n_cube_edge=10, 
                                    ninter = 500, dt = 0.001, flag_compile=False)

N-Body (C) Flavio Manoel, v1.5.5
N = 1000 corpos, dt = 0.001s, M = 500 interações
Ei = -6221418016861.29, Ki = 0.0, Ui = -6221418016861.29
Após 500 passos, 0.5000s :
Ef = -6221859489860.984, Kf = 211027971915.9048, Uf = -6432887461776.889, erro em E = 0.0070960189220183505%
Código executado em 23.631115 s


### Teste de nbody em Python Numba CPU

In [35]:
t_all_nb, t_nb, erro_nb = run_nbody(gravitationalNbody_NumbaCPU, n_cube_edge=10, ninter = 500, dt = 0.001)

N-Body (C) Flavio Manoel, v1.5.5
N = 1000 corpos, dt = 0.001s, M = 500 interações
Após 500 passos, 0.5000s, Ef = -6221859489860.983398, Kf = 211027971915.905273, Uf = -6432887461776.888672
Código executado em 2.711705 s

N-Body (C) Flavio Manoel, v1.5.5
N = 1000 corpos, dt = 0.001s, M = 500 interações
Ei = -6221418016861.29, Ki = 0.0, Ui = -6221418016861.29
Após 500 passos, 0.5000s :
Ef = -6221859489860.983, Kf = 211027971915.90527, Uf = -6432887461776.889, erro em E = 0.007096018922002654%
Código executado em 2.330788 s


### Teste de nbody em NumPy Numba CPU

In [36]:
t_all_np_nb, t_np_nb, erro_np_nb = run_nbody(gravitationalNbody_NumPy_NumbaCPU, n_cube_edge=10, 
                                             ninter = 500, dt = 0.001)

N-Body (C) Flavio Manoel, v1.5.5
N = 1000 corpos, dt = 0.001s, M = 500 interações
Após 500 passos, 0.5000s, Ef = -6221859489860.983398, Kf = 211027971915.905273, Uf = -6432887461776.888672
Código executado em 9.067359 s

N-Body (C) Flavio Manoel, v1.5.5
N = 1000 corpos, dt = 0.001s, M = 500 interações
Ei = -6221418016861.29, Ki = 0.0, Ui = -6221418016861.29
Após 500 passos, 0.5000s :
Ef = -6221859489860.983, Kf = 211027971915.90527, Uf = -6432887461776.889, erro em E = 0.007096018922002654%
Código executado em 7.928783 s


### Teste de nbody em Python Numba CPU //

In [37]:
t_all_nb_parallel, t_nb_parallel, erro_nb_parallel = run_nbody(gravitationalNbody_NumbaCPUparallel, 
                                                               n_cube_edge=10, ninter = 500, dt = 0.001)

N-Body (C) Flavio Manoel, v1.5.5
N = 1000 corpos, dt = 0.001s, M = 500 interações
Após 500 passos, 0.5000s, Ef = -6221841961970.263672, Kf = 211054625605.324768, Uf = -6432896587575.588867
Código executado em 0.416550 s

N-Body (C) Flavio Manoel, v1.5.5
N = 1000 corpos, dt = 0.001s, M = 500 interações
Ei = -6221418016867.902, Ki = 0.0, Ui = -6221418016867.902
Após 500 passos, 0.5000s :
Ef = -6221841962056.772, Kf = 211054625695.62173, Uf = -6432896587752.395, erro em E = 0.006814285549061165%
Código executado em 0.413628 s


### Teste de nbody em NumPy Numba CPU //

In [38]:
t_all_np_nb_parallel, t_np_nb_parallel, erro_np_nb_parallel = run_nbody(
                            gravitationalNbody_NumPy_NumbaCPUparallel, n_cube_edge=10, ninter = 500, dt = 0.001)

N-Body (C) Flavio Manoel, v1.5.5
N = 1000 corpos, dt = 0.001s, M = 500 interações
Após 500 passos, 0.5000s, Ef = -6221859489861.064453, Kf = 211027971915.904785, Uf = -6432887461776.969727
Código executado em 6.163906 s

N-Body (C) Flavio Manoel, v1.5.5
N = 1000 corpos, dt = 0.001s, M = 500 interações
Ei = -6221418016867.902, Ki = 0.0, Ui = -6221418016867.902
Após 500 passos, 0.5000s :
Ef = -6221859489861.064, Kf = 211027971915.9048, Uf = -6432887461776.97, erro em E = 0.007096018817015025%
Código executado em 1.421471 s


### Resultados e comparações

Speed up of pure Python respect to Python (with NumPy arrays)  :

In [39]:
t_py_a / t_py

4.3322128112051965

Speed up of Python NumPy respect to pure Python :

In [40]:
t_py / t_np

13.235447240345593

Speed up of Python Numba CPU (serial) with respect to pure Python :

In [41]:
t_py / t_nb

134.18999828526964

Speed up of Python Numba CPU (serial) with respect to Python (with NumPy arrays) :

In [42]:
t_py_a / t_nb

581.3396297070484

Speed up of NumPy Numba CPU (serial) with respect to Python NumPy :

In [43]:
t_np / t_np_nb

2.980421359917387

Speed up of Python Numba CPU // with respect to pure Python :

In [44]:
t_py / t_nb_parallel

756.1585696769041

Speed up of Python Numba CPU // with respect to Python (with NumPy arrays) :

In [45]:
t_py_a / t_nb_parallel

3275.839842856881

Speed up of NumPy Numba CPU // with respect to Python NumPy :

In [46]:
t_np / t_np_nb_parallel

16.62440358204243

Speed up of Python Numba CPU // with respect to Python Numba CPU :

In [47]:
t_nb / t_nb_parallel  # speed-up vs. Numba CPU function

5.634984569188339

Speed up of NumPy Numba CPU // with respect to NumPy Numba CPU :

In [48]:
t_np_nb / t_np_nb_parallel  # speed-up vs. Numba CPU function

5.577870231913528